----

# CMSE401 Quiz Instructions

This quiz is designed to take approximately 30 minutes to complete (you will be given the entire class period).  

Please read the following instructions before starting the quiz.


> This is an open Internet quiz.  Feel free to use anything on the Internet with one important exception...
> 
> - **DO NOT** communicate live with other people or AI tools during the quiz (either verbally or on-line).  The goal here is to find answers to problems as you would in the real world and demonstrate your own ability to solve problems.  
> 
> You will be given **until the end of class** to complete this quiz.  Use your time wisely. 
> 
> **HINTS:**
> - Neatness and grammar are important.  We will ignore all notes or code we can not read or understand.
> - Read the entire quiz from beginning to end before starting.  Not all questions are equal in **points vs. time** so plan your time accordingly. 
> - Spaces for answers are provided. Delete the prompting text such as "Put your answer to the above question here" and replace it with your answer. Do not leave the prompting text with your answer.
> - Do not assume that the answer must be in the same format of the cell provided. Feel free to change the cell formatting (e.g., markdown to code, and vice versa) or add additional cells as needed to provide your answer.
> - When a question asks for an answer "**in your own words**" it is still okay to search the Internet for the answer as a reminder. *However*, we would like you to do more than cut and paste.  Make the answer your own. 
> - If you get stuck, try not to leave an answer blank. It is better to include some notes or stub functions so we have an idea about your thinking process so we can give you partial credit.   
> - Always provid links to any references you find helpful. 
> - Feel free to delete the provided check marks (&#9989;) as a way to keep track of which questions you have successfully completed. 

> **Honor Code**
> 
> I, agree to neither give nor receive any help on this quiz from other people.  I also understand that providing answers to questions on this quiz to other students is also an academic misconduct violation as is live communication or receiving answers to questions on this quiz from other people. It is important to me to be a person of integrity and that means that ALL ANSWERS on this quiz are my answers.
> 
> &#9989; **<font color=red>DO THIS:</font>** Include your name in the line below to acknowledge the above statement:

Put your name here.

---

# Yet Another Pi Code


It turns out there are many different ways to approximate pi. We used one such method as an example in several classes. Here we will explore another example of computing pi. This version computes pi by simulating throwing darts at a 1x1 board (x: [0,1], y: [0,1]) and computing the fraction that falls on or within radius 1 of the origin. That fraction is then multiplied by 4 to estimate pi. 

While the version we explored in class was very good at finding a quality estimate with few iterations, this version is highly sensitive to the number of iterations and requires *many* iterations to arrive at a good estimate, therefore making it a good target for parallel computing. 

The serial version for the pi code in C is below

```c
#include "stdio.h"
#include <math.h>
#include <stdlib.h>
static long num_steps = 10000000;
double step;

int main(){
    double pi,sum;
    int steps = num_steps;
    sum=0.0;
    unsigned int seed = 1;
    for (int i=0;i<steps;i++) {
        double val=sqrt((double) pow((double) rand_r(&seed)/ RAND_MAX,2.0)+ (double) pow((double) rand_r(&seed)/ RAND_MAX,2.0));
        if (val<=1.0)
                sum= sum+1.0;
        }


    pi = 4.0*sum/(double) num_steps;
    printf("%f\n\n",pi);
}
```

&#9989; **<font color=red>Question 1</font>**: (10 points) First, log into a dev node on the HPCC. Copy the serial code into a C file and then compile and run the file to test it out. Write the code below that you used to compile and run the C code. 

I used a makefile

```make
all: pi

pi: pi.c
    gcc -lm -o pi pi.c

clean:
    rm pi.out
```

&#9989; **<font color=red>Question 2</font>**: (10 points) Now that you have the C code running on the HPCC, we want to get a baseline measure of performance. Run the code several times using the default `num_steps` and record the average runtime. Include both the average runtime and code used to measure the runtime below. 

The average runtime was 0.698 seconds. I used a script I previouslt wrote called `time10` shown below

```bash
export TIMEFORMAT=%R;

total=0;
for i in {1..10}; do
    time=$((time $@) 2>&1);
    total=$(echo "$total + $time" | bc -l)
done;
avg=$(echo "$total/10" | bc -l)
echo $avg
```

&#9989; **<font color=red>Question 3</font>**: (20 points) Now we want to parallelize the code using OpenMP. Copy the supplied serial code into a new C file and modify/optimize the code so that it can leverage multiple threads. Set the number of threads to 10. Paste your modified C code below. 


```C
#include "stdio.h"
#include <math.h>
#include <stdlib.h>
#include <omp.h>
static long num_steps = 10000000;
double step;

int main(){
    double pi,sum;
    int steps = num_steps;
    sum=0.0;
    unsigned int seed = 1;
    omp_set_num_threads(10);
    #pragma omp parallel for reduction(+:sum)
    for (int i=0;i<steps;i++) {
        double val=sqrt((double) pow((double) rand_r(&seed)/ RAND_MAX,2.0)+ (double) pow((double) rand_r(&seed)/ RAND_MAX,2.0));
        if (val<=1.0)
                sum=sum+1.0;
        }


    pi = 4.0*sum/(double) num_steps;
    printf("%f\n\n",pi);
}
```

&#9989; **<font color=red>Question 4</font>**: (10 points) Now lets compile and run the new parallel version. Record the runtime several times to get an average runtime. Record the code used to compile and run your parallel code as well as the average runtime. 

```make
all: pi-p

pi: pi.c
    gcc -lm -o pi pi.c

pi-p:
    gcc -lm -fopenmp -o pi-p pi-p.c

clean:
    rm pi.out
    rm pi-p.out

```

Average runtime for 10 runs: 0.0128

&#9989; **<font color=red>Question 5</font>**: (10 points) Did your code run faster or slower than the serial version? If it ran faster, explain what design choices you made to ensure it would run faster. If it ran slower, explain why you think this is the case. 

It ran faster. I made sure to include the `seed` as a thread-local variable so that it would not need to access a shared variable within the parallel threads.

&#9989; **<font color=red>Question 6</font>**: (10 points) Take note of the line `unsigned int seed = 1`. This line sets the seed needed for the random number generator. Play around with the placement of this line and record any differences in runtime performance and the quality of the pi estimate. Try the following
    - place the line before the parallel region
    - place the line inside the parallel region but not in the for loop
    - place the line inside the for loop

What did you notice when moving this line around? Explain why you think this behavior occured in the different cases. 

When outside the parallel region the parallel code is slower than serial. This is likely because each thread will have to wait for the others in order to access the shared variable so we are not really doing parallel computing but we have the extra overhead of creating threads.

When in the parallel region but outside the for loop we get a significant speed-up. This is because we actually get to do parallel computing since the threads don't have to wait to access a shared variable.

When in the for loop we get fast code, but the estimate is completely off. This is sort of strance behavior, but I would expect it has something to do with the way c generates random numbers. We are using the address of the seed variable as the seed, so resetting it in each loop changes the seed each time as the local variable is created and destroyed each loop.

&#9989; **<font color=red>Question 7</font>**: (10 points) Finally, lets setup a SLURM submission script. In this job script, do the following:

- set the allocated time to 5 minutes
- set the number of nodes to 1
- set the number of cores to 10
- set the memory to 1 Gb
- set the job name to CMSE401
- finally, include the code used to run and benchmark both the parallel and serial pi code

Once completed, copy and paste the contents of the submission script below. As well, submit the job and paste the code used to submit the job. 


```batch


```

```bash
#!/bin/bash --login

#SBATCH --time=00:05:00
#SBATCH -N 1
#SBATCH --cpus-per-task=10
#SBATCH --mem=1G
#SBATCH --job-name=CMSE401

time10 pi

time10 pi-p.out
```

To run use `sbatch pi.sb`

# Congratulations

You are done with your quiz. Please save the file and upload the jupyter notebook and any other necessary files to the D2L dropbox. 

Written by Dr. Nathan Haut, Michigan State University
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.

----